In [ ]:
#引包，dct特征用opencv中的方法来提取。
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time
from __future__ import division

In [ ]:
#读取训练集图片和标签
train_img = np.load('train.pkl')
train_label = np.loadtxt('train.csv',delimiter=',')

In [ ]:
#定义单张图片dct特征的提取方法
def dct(img):
    img = np.float64(img)/255.0
    out = np.zeros((1,3,8,8),dtype=np.float64)
    for i in xrange(img.shape[0] // 2 - 3):
        for j in xrange(img.shape[1] // 2 - 3):
            r = img[2*i:2*i+8,2*j:2*j+8,0]
            g = img[2*i:2*i+8,2*j:2*j+8,1]
            b = img[2*i:2*i+8,2*j:2*j+8,2]
            r_ = cv2.dct(r).reshape(1,1,8,8)
            g_ = cv2.dct(g).reshape(1,1,8,8)
            b_ = cv2.dct(b).reshape(1,1,8,8)
            temp = np.concatenate((r_,g_,b_),axis = 1)
            out = np.concatenate((out,temp),axis=0)
    out = out[1:]
    return out

In [ ]:
#特征降维（从192降至63）
def down_size(img):
    out = img.reshape(5673,3,-1)[:,:,[0,1,2,3,4,5,8,9,10,11,12,16,17,18,19,24,25,26,32,33,40]].reshape(5673,-1)
    return out

In [ ]:
#定义单张图片的GMM聚类方法
def init_params(initU,X,K):
    N,D = X.shape
    U = initU
    A = np.zeros(K)
    E = np.zeros((K,D,D))
    distmat = np.tile(np.sum(X * X,axis=1),(K,1)).T \
    + np.tile(np.sum(U * U,axis = 1).T,(N,1)) \
    - 2 * np.dot(X,U.T)
    labels = np.argmin(distmat,axis=1)

    for k in range(K):
        Xk = X[labels==k]
        A[k] = float(np.shape(Xk)[0]) / N
        E[k] = np.cov(Xk.T)
    return U,E,A

def GMM(arrs,gnum):
    #初始化
    N,dim = arrs.shape
    randlist = np.random.choice(N, gnum, replace=False)
    U = np.copy(arrs[randlist])
    U,E,A = init_params(U,arrs,gnum)
    E_d = np.eye(dim)*5e-4
    r = np.zeros((gnum,N))
    lh = 0
    olh = 1
    
    while np.abs(lh - olh)> 1e-100:
        
        olh = lh
        
        #（E步）
        for i in range(gnum):
            det_E = np.linalg.det(E[i]+E_d)
            inv_E = np.linalg.inv(E[i]+E_d)
            xishu = 1 / (det_E ** 0.5 * (2.0 * np.pi) ** (dim/2))
            X_U = arrs - U[i]
            zhishu = np.sum(np.dot(X_U,inv_E) * X_U,axis=1) *(-0.5)
            r[i] = xishu * np.exp(zhishu)
        prob = r
        r = prob * A[:,np.newaxis]
        r /= np.sum(r,axis=0)
                
        #（M步）
        A = np.sum(r,axis=1)
        
        U = np.dot(r,arrs)/A[:,np.newaxis]
        
        for i in range(gnum):
            t = arrs - U[i]
            E[i] = np.dot(t.T*r[i],t)
            
        E /= A[:,np.newaxis,np.newaxis]
            
        A /= N
            
        lh = np.sum(np.log(np.sum(prob*A[:,np.newaxis],axis=0)))
        
    return (U,E,A)

In [ ]:
#得到每张图片的GMM模型,模型为GMM-8
tic = time.time()
K = 8
ueas = []
count = 0
for img in train_img:
    ueas.append(GMM(down_size(dct(img)),K))
    count += 1
    if count%10 == 0:
        print count,': ',time.time() - tic

In [ ]:
#将ueas进行一些处理，变为Us,Es,As，并保存中间结果，以备之后使用。
def ueas_change(ueas):
    Us = np.zeros((4500,8,63))
    Es = np.zeros((4500,8,63,63))
    As = np.zeros((4500,8))
    for i in xrange(4500):
        Us[i] = ueas[i][0]
        Es[i] = ueas[i][1]
        As[i] = ueas[i][2]
    return Us,Es,As
Us,Es,As = ueas_change(ueas)
np.savez_compressed('ueas',U=Us,E=Es,A=As)